In [1]:
import pandas as pd

### 데이터 베이스 설정

In [2]:
DB_URL = 'onad.cbjjamtlar2t.ap-northeast-2.rds.amazonaws.com'
DB_USER = 'onad'
DB_PASSWORD = 'rkdghktn12'
DB_DATABASE = 'onad'
DB_CHARSET = 'utf8mb4'
DB_LOGFLAG  = 'False'
DB_PORT = 3306

db_url = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    DB_USER, DB_PASSWORD,
    DB_URL, DB_PORT,
    DB_DATABASE, DB_CHARSET
)

In [3]:
def data_from_sql_twitchchat(streamer, target_date):
    query = """
        select * from twitch_chat where streamer_name = "%s" and broad_date ="%s"
    """% (streamer, target_date)

    return pd.read_sql_query(query, db_url)

### 채팅 데이터 로드

In [440]:
streamer = "rhdgurwns"
target_date = "2018-12-09"
chat_data = data_from_sql_twitchchat(streamer, target_date)

### Exploratory Data Analysis

In [441]:
chat_data.shape

(57597, 6)

In [442]:
chat_data.head()

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
0,5804896,rhdgurwns,2018-12-09,<명부러전 (worldbest29)>,08:02:31,공혁준 방송켜라
1,5804897,rhdgurwns,2018-12-09,<명부러전 (worldbest29)>,08:02:39,방송켜 뽀뽀 쪽
2,5804898,rhdgurwns,2018-12-09,<럼주 (geonw90)>,09:12:19,오뱅없?
3,5804899,rhdgurwns,2018-12-09,<@%rhdqurwns1>,09:18:23,없을수도
4,5804900,rhdgurwns,2018-12-09,<@%rhdqurwns1>,09:18:34,방송에서 자세히 말을안하니까..


In [443]:
print("채팅 로그의 수: %s" % (chat_data.shape[0]))
print("순수 시청자 수: %s" % (len(list(chat_data['chatterer'].unique()))))

채팅 로그의 수: 57597
순수 시청자 수: 4950


In [444]:
print("채팅로그의 시작과 끝 : %s ~ %s " % (sorted(chat_data['chat_time'])[0], sorted(chat_data['chat_time'])[-1]))

채팅로그의 시작과 끝 : 08:02:31 ~ 22:18:28 


### 시간데이터를 DatetimeIndex 로 변경

In [110]:
chat_data['time'] = pd.to_datetime(
    chat_data['broad_date'].apply(lambda x : ",".join(x.split("-"))) + "," + \
    chat_data['chat_time'].apply(lambda x : ",".join(x.split(":"))), 
                                  format="%Y,%m,%d,%H,%M,%S")


In [111]:
chat_data_reindex = chat_data.set_index('time')

In [112]:
chat_data_reindex.head(2)

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
time,,,,,,
2018-12-27 17:43:26,5235567,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:26,오늘방송이 안켜진다면
2018-12-27 17:43:36,5235568,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:36,아마도 뜨겁겠지..?


### pd.Datetime index 생성

In [113]:
# YYYY-mm-dd HH:MM:SS 의 datetime 객체 인덱스 생성 (00:00:00 ~ 23:59:59 까지)
index=pd.date_range(target_date, periods=86400, freq="S")
# 인덱스만 가지는 데이터 프레임 생성 (chat_data와 병합하기 위함)
df = pd.DataFrame(index=index,)

In [114]:
# 필요없는 행 삭제
del chat_data_reindex['chat_id']
del chat_data_reindex['streamer_name']
del chat_data_reindex['broad_date']
del chat_data_reindex['chat_time']

In [115]:
chat_data_reindex.head(2)

,chatterer,chat_contents
time,,
2018-12-27 17:43:26,<$Lee_sang_hyuk>,오늘방송이 안켜진다면
2018-12-27 17:43:36,<$Lee_sang_hyuk>,아마도 뜨겁겠지..?


### 한 문장에서 특정 단어의 수 세어 컬럼으로 생성

In [361]:
chat_data_reindex['cnt_ㅋ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㅋ"))
chat_data_reindex['cnt_ㄵ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄵ"))
chat_data_reindex['cnt_ㄴㅈ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄴㅈ"))
chat_data_reindex['cnt_오'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("오"))
chat_data_reindex['cnt_와'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("와"))
chat_data_reindex['cnt_유하'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("유하"))
chat_data_reindex['cnt_수금'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("수금"))


In [362]:
chat_data_reindex.head()

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ,cnt_오,cnt_와,cnt_유하,cnt_수금
time,,,,,,,,,
2018-12-27 17:43:26,<$Lee_sang_hyuk>,오늘방송이 안켜진다면,0,0,0,1,0,0,0
2018-12-27 17:43:36,<$Lee_sang_hyuk>,아마도 뜨겁겠지..?,0,0,0,0,0,0,0
2018-12-27 17:43:45,<rnwhdan>,오뱅없?,0,0,0,1,0,0,0
2018-12-27 18:17:10,<%당근빠따 (ambus000)>,looksamJun looksamJun looksamJun,0,0,0,0,0,0,0
2018-12-27 18:31:22,<rnwhdan>,절 대 방 송 켜,0,0,0,0,0,0,0


### 해당 채팅 수 빈도를 갖는 데이터 프레임 생성

In [394]:
pivot_df = chat_data_reindex.pivot_table(index=chat_data_reindex.index, aggfunc=len, values='chatterer')
pivot_df.columns = ['cnt_chat']  # 컬럼 이름 할당
pivot_df["cnt_ㅋ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㅋ')['cnt_ㅋ']
pivot_df["cnt_ㄵ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄵ')['cnt_ㄵ'] +\
                    chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄴㅈ')['cnt_ㄴㅈ']
pivot_df["cnt_오"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_오')['cnt_오']
pivot_df["cnt_와"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_와')['cnt_와']
pivot_df["cnt_유하"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_유하')['cnt_유하']
pivot_df["cnt_수금"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_수금')['cnt_수금']


# 초당 채팅수가 평균 이상인 경우만 보이도록
pivot_df[pivot_df['cnt_chat'] > pivot_df['cnt_chat'].mean()]

print(sorted(pivot_df['cnt_chat'].unique(), reverse=True)[:10])
print(sorted(pivot_df['cnt_ㅋ'].unique(), reverse=True)[:10])
print(sorted(pivot_df['cnt_수금'].unique(), reverse=True))

pivot_df[pivot_df['cnt_수금'] == 6]  # 해당 글자가 가장 높은 구간

[39, 38, 36, 33, 32, 31, 28, 27, 26, 25]
[371, 334, 316, 302, 296, 267, 264, 251, 248, 238]
[6, 4, 3, 2, 1, 0]


,cnt_chat,cnt_ㅋ,cnt_ㄵ,cnt_오,cnt_와,cnt_유하,cnt_수금
time,,,,,,,
2018-12-27 20:20:02,22,71,0,0,0,1,6


### 해당 글자의 빈도수가 높은 구간의 채팅

In [395]:
chat_data_reindex[chat_data_reindex.index == "2018-12-27 20:20:02"]

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ,cnt_오,cnt_와,cnt_유하,cnt_수금
time,,,,,,,,,
2018-12-27 20:20:02,<$wonseok1827>,???????????????,0,0,0,0,0,0,0
2018-12-27 20:20:02,<라디보바2 (rlaalstj2104)>,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,36,0,0,0,0,0,0
2018-12-27 20:20:02,<717l7i>,월수입,0,0,0,0,0,0,0
2018-12-27 20:20:02,<hwahyunn>,수 금 박 사,0,0,0,0,0,0,0
2018-12-27 20:20:02,<반짝콩 (mightybeenflash)>,개역겹다 ㄹㅇ,0,0,0,0,0,0,0
2018-12-27 20:20:02,<예절주입기__ (jsho1313)>,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,29,0,0,0,0,0,0
2018-12-27 20:20:02,<얍얍씨밥세끼챙겨드세요 (didgus486)>,유하,0,0,0,0,0,1,0
2018-12-27 20:20:02,<알딱이 (alsdud3162)>,스금한다,0,0,0,0,0,0,0
2018-12-27 20:20:02,<Tain1218>,밍,0,0,0,0,0,0,0


### 시작시간 불러와 시작시간으로부터 00:00:00 으로 시작하는 컬럼만들기

In [396]:
streamer_id = 103861159
query = "select * from twitch_stream where streamer_id = '%s'" % streamer_id
broad_date_df = pd.read_sql_query(query, db_url)

broad = pd.to_datetime(broad_date_df['broad_date'])
broad.apply(lambda x : x + pd.Timedelta("9hours"))  # 트위치 시간대는 UTC 기준이므로 한국시간으로 +9 하여 변경

0   2019-01-05 18:09:46
1   2019-01-07 18:04:45
Name: broad_date, dtype: datetime64[ns]

### 인덱스를 00 부터 시작

In [397]:
# 데이터프레임 복사
index_new_df = chat_data_reindex.copy()
index_new_df.index[0]

Timestamp('2018-12-27 17:43:26')

In [398]:
# 방송시작시간이 다음과 같다면
start_time = "2018-12-27 18:53:50"
start_time = pd.to_datetime(start_time)


In [399]:
# 첫 채팅시간
first_chat_time = chat_data_reindex[chat_data_reindex.index == "2018-12-27 17:43:26"].index.unique()
first_chat_time

DatetimeIndex(['2018-12-27 17:43:26'], dtype='datetime64[ns]', name='time', freq=None)

In [400]:
# 첫 채팅 시간 - 방송 시작 시간 을 통해 방송시작시간으로부터 몇초가 지나고 있는지 확인 가능
first_chat_time - start_time

TimedeltaIndex(['-1 days +22:49:36'], dtype='timedelta64[ns]', name='time', freq=None)

In [401]:
# 첫 채팅 시간 - 방송 시작시간을 인덱스로
index_new_df['korea_time'] = index_new_df.index
index_new_df.index = index_new_df['korea_time'].apply(lambda x : x - start_time)
index_new_df.index.name = "stream_time"

In [402]:
# 인덱스 중 방송 시작 이전의 데이터는 제외한 나머지
new_df = index_new_df[index_new_df.index >= "00:00:00"]

### 인덱스를 기준으로 피봇팅

In [403]:
# 채팅 빈도수 데이터 준비
new_pivot_df1 = new_df.pivot_table(index=new_df.index, values=['chat_contents'], aggfunc=len)

# 이외의 빈도수 데이터 준비
new_pivot_df2 = new_df.pivot_table(index=new_df.index,
                                   values=['cnt_ㅋ','cnt_ㄵ','cnt_ㄴㅈ','cnt_오', 'cnt_와', 'cnt_유하'],
                                  aggfunc=sum)

In [411]:
anal_df = new_pivot_df1.merge(new_pivot_df2, on=new_pivot_df1.index)
anal_df.set_index('key_0', inplace=True)
anal_df.index.name = "stream_time"


Index(['chat_contents', 'cnt_ㄴㅈ', 'cnt_ㄵ', 'cnt_ㅋ', 'cnt_오', 'cnt_와',
       'cnt_유하'],
      dtype='object')

In [407]:
anal_df.head()

,chat_contents,cnt_ㄴㅈ,cnt_ㄵ,cnt_ㅋ,cnt_오,cnt_와,cnt_유하
stream_time,,,,,,,
00:00:02,1,0,0,3,0,1,0
00:00:03,1,0,0,0,0,0,0
00:00:04,6,0,0,1,3,0,0
00:00:05,1,0,0,0,0,0,0
00:00:06,1,0,0,0,0,0,0


### 검증데이터를 추가, 검증데이터와 분석데이터간 상관관계를 알아보자

- 검증데이터: yes/no 의 명목형 변수  
- 이외의 다른 열 : 연속형 변수  
명목형 변수와 연속형 변수의 상관관계의 경우 **`Point-biserial correlation`** 을 사용함  
이는 scipy.stats 모듈에 pointbiserialr 메소드로 구현되어 있음

#### 시험해보기 위한 치팅
- 채팅 수가 60% 중위수 이상인 경우 모두 하이라이트영상이 있다고 가정하여 데이터프레임 생성

In [435]:
import numpy as np
percentile = np.percentile(anal_df.chat_contents.unique(), 70)
anal_df[anal_df['chat_contents'] > percentile_60].head(5)

,chat_contents,cnt_ㄴㅈ,cnt_ㄵ,cnt_ㅋ,cnt_오,cnt_와,cnt_유하,validation
stream_time,,,,,,,,
00:20:49,22,0,0,5,3,3,0,1
00:36:25,21,0,0,17,0,0,0,1
00:37:10,22,0,0,8,1,0,0,1
00:37:12,25,0,0,14,0,0,0,1
00:37:31,23,0,0,248,0,0,0,1


In [436]:
anal_df['validation'] = anal_df['chat_contents'].apply(lambda x : 1 if x >= percentile else 0)

In [437]:
anal_df['validation'].unique()

array([0, 1], dtype=int64)

#### scipy.stats 불러오기

In [438]:
from scipy import stats
for col in anal_df:
    print("%s 상관 정도 : " % col, stats.pointbiserialr(anal_df[col], anal_df['validation']).correlation)

chat_contents 상관 정도 :  0.23198985901377472
cnt_ㄴㅈ 상관 정도 :  -0.0016685854538941738
cnt_ㄵ 상관 정도 :  0.009108262095028404
cnt_ㅋ 상관 정도 :  0.08650590461405516
cnt_오 상관 정도 :  0.019138265202727735
cnt_와 상관 정도 :  0.06473002579664976
cnt_유하 상관 정도 :  0.02237208866731631
validation 상관 정도 :  1.0


In [439]:
anal_df.corr()

,chat_contents,cnt_ㄴㅈ,cnt_ㄵ,cnt_ㅋ,cnt_오,cnt_와,cnt_유하,validation
chat_contents,1.000000,0.019593,0.051020,0.515119,0.129426,0.204489,0.075268,0.231990
cnt_ㄴㅈ,0.019593,1.000000,0.193046,-0.003860,-0.007517,0.001845,-0.002067,-0.001669
cnt_ㄵ,0.051020,0.193046,1.000000,0.009728,0.019842,0.070790,0.002200,0.009108
cnt_ㅋ,0.515119,-0.003860,0.009728,1.000000,0.023803,0.071581,0.046684,0.086506
cnt_오,0.129426,-0.007517,0.019842,0.023803,1.000000,0.068881,-0.005094,0.019138
cnt_와,0.204489,0.001845,0.070790,0.071581,0.068881,1.000000,0.016995,0.064730
cnt_유하,0.075268,-0.002067,0.002200,0.046684,-0.005094,0.016995,1.000000,0.022372
validation,0.231990,-0.001669,0.009108,0.086506,0.019138,0.064730,0.022372,1.000000


### 수집한 검증데이터로 상관관계알아보자
